# QML Tutorial

by Anders S. Christensen.

This tutorial is a general introduction to kernel-ridge regression with Quantum Machine Learning (QML).

QML is freely available under the terms of the MIT open-source license. However, we ask that you properly cite the relevant, underlying work in your published work (provided below).

##### QML Citation: 
AS Christensen, FA Faber, B Huang, LA Bratholm, A Tkatchenko, KR Müller, OA von Lilienfeld (2017) "QML: A Python Toolkit for Quantum Machine Learning" https://github.com/qmlcode/qml

##### Theory Citation:
[Rupp et al, Phys Rev Letters, 2012](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.058301)

[Montavon et al, New J Phys, 2013](http://iopscience.iop.org/article/10.1088/1367-2630/15/9/095003/meta)


## Theory 
Regression model of some property, 

\begin{equation*}
y
\end{equation*}

for some system, 

\begin{equation*}
\widetilde{\mathbf{X}}
\end{equation*}

this could correspond to e.g. the atomization energy of a molecule:

\begin{equation*}
y\left(\widetilde{\mathbf{X}} \right) = \sum_i \alpha_i \  K\left( \widetilde{\mathbf{X}}, \mathbf{X}_i\right)
\end{equation*}

E.g. Using Gaussian kernel function with Frobenius norm:

\begin{equation*}
K_{ij} = K\left( \mathbf{X}_i, \mathbf{X}_j\right) = \exp\left( -\frac{\| \mathbf{X}_i - \mathbf{X}_j\|_2^2}{2\sigma^2}\right)
\end{equation*}

Regression coefficients are obtained through kernel matrix inversion and multiplication with reference labels

\begin{equation*}
\boldsymbol{\alpha} = (\mathbf{K} + \lambda \mathbf{I})^{-1} \mathbf{y}
\end{equation*}

## Load QML into Jupyter Notebook

In [0]:
!sudo apt-get install python-pip gfortran libblas-dev liblapack-dev git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
liblapack-dev is already the newest version (3.7.1-4ubuntu1).
gfortran is already the newest version (4:7.4.0-1ubuntu2.3).
git is already the newest version (1:2.17.1-1ubuntu0.5).
The following additional packages will be installed:
  libblas3 libpython-all-dev python-all python-all-dev python-asn1crypto
  python-cffi-backend python-crypto python-cryptography python-dbus
  python-enum34 python-gi python-idna python-ipaddress python-keyring
  python-keyrings.alt python-pip-whl python-pkg-resources python-secretstorage
  python-setuptools python-six python-wheel python-xdg
Suggested packages:
  liblapack-doc python-crypto-doc python-cryptography-doc
  python-cryptography-vectors python-dbus-dbg python-dbus-doc
  python-enum34-doc python-gi-cairo gnome-keyring libkf5wallet-bin
  gir1.2-gnomekeyring-1.0 python-fs python-gdata python-keyczar
  python-secretstorage-doc python-setuptools-doc
The fo

In [0]:
!pip install qml # --user -U

     |████████████████████████████████| 51kB 2.9MB/s 
  Created wheel for qml: filename=qml-0.4.0.27-cp36-cp36m-linux_x86_64.whl size=976073 sha256=f5bbfc638b9d5de247ffd3b05321cbe9826923df9d166a36d221300dbd211dd4
  Stored in directory: /root/.cache/pip/wheels/ed/12/cf/9f6f875260ccc47dbdbd131631c416e24c933c84a8a20e2bc8
Successfully built qml


In [0]:
from __future__ import print_function

import sys
print("Printing version info for help reporting bugs")
print("Python version:", sys.version)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

try:
    import qml
    from qml.kernels import gaussian_kernel
    from qml.kernels import laplacian_kernel
    from qml.math import cho_solve
    print("QML version:",qml.__version__)
except ImportError:
    print("Failed to find QML")
    print("Please follow instructions here: http://www.qmlcode.org/installation.html")


Printing version info for help reporting bugs
Python version: 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
QML version: 0.2.1


In [0]:
!git clone https://github.com/qmlcode/tutorial.git

Cloning into 'qmlcode'...
remote: Not Found
fatal: repository 'https://github.com/qmlcode.git/' not found


In [0]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Tutorial exercises

If you got this notebook without the repo, please clone the following GIT repository to access the necessary scripts and QM7 dataset (atomization energies and relaxed geometries at PBE0/def2-TZVP level of theory) for ~7k GDB1-7 molecules. 
[Ruddigkeit et al, J Chem Inf Model, 2012](https://pubs.acs.org/doi/abs/10.1021/ci300415d)
[Rupp et al, Phys Rev Letters, 2012](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.058301)

git clone https://github.com/qmlcode/tutorial.git

Additionally, the repository contains Python3 scripts with the solutions to each exercise.

# Exercise 1: Representations

In this exercise we use QML to generate the Coulomb matrix and Bag of bonds (BoB) representations. [Montavon et al, New J Phys, 2013](http://iopscience.iop.org/article/10.1088/1367-2630/15/9/095003/meta)
In QML data can be parsed via the <font color=red>Compound</font> class, which stores data and generates representations in Numpy's ndarray format.
If you run the code below, you will read in the file <font color=red>qm7/0001.xyz</font> (a methane molecule) and generate a coulomb matrix representation (sorted by row-norm) and a BoB representation.

In [0]:
sys.path.append('/content/drive/My Drive')
path = "drive/My Drive/Master/Kaishi/qml/xyz/"
sys.path.append('/content/tutorial')

In [0]:
# Create the compound object mol from the file qm7/0001.xyz which happens to be methane
mol = qml.Compound(xyz=path+str(0)+"+0.xyz")

# Generate and print a coulomb matrix for compound with 5 atoms 
mol.generate_coulomb_matrix(size=21, sorting="row-norm")
print(mol.representation)

# Generate and print BoB bags for compound containing C and H
#mol.generate_bob(asize={"C":2, "H":5})
#print(mol.representation)

NameError: ignored

The representations are simply stored as 1D-vectors.
Note the keyword <font color=red>size</font> which is the largest number of atoms in a molecule occurring in test or training set. 
Additionally, the coulomb matrix can take a sorting scheme as keyword, and the BoB representations requires the specifications of how many atoms of a certain type to make room for in the representations.

Lastly, you can print the following properties which is read from the XYZ file:

In [0]:

sys.path.append('/content/tutorial')
from tutorial.tutorial_data import compounds

FileNotFoundError: ignored

In [0]:
!cp drive/"My Drive"/Master/Kaishi/qml/qml_dr_obabel.py .
!cp drive/"My Drive"/Master/Kaishi/qml/obabel_dG.txt .
from qml_dr_obabel import compounds

In [0]:
#!rm qml_dr_obabel.py

In [0]:
# Print other properties stored in the object
print(mol.coordinates)
print(mol.atomtypes)
print(mol.nuclear_charges)
print(mol.name)
print(mol.unit_cell)

In [0]:
compounds[0].name, compounds[0].properties
#compounds

('drive/My Drive/Master/Kaishi/qml/obabel-xyz/2185+0.xyz', 19.802231799748792)

# Exercise 2: Kernels

In this exercise we generate a Gaussian kernel matrix, 

\begin{equation*}
\mathbf{K}
\end{equation*}

using the representations

\begin{equation*}
\mathbf{X}
\end{equation*}

which are generated similarly to the example in the previous exercise:

\begin{equation*}
K_{ij} = \exp\left( -\frac{\| \mathbf{X}_i - \mathbf{X}_j\|_2^2}{2\sigma^2}\right)
\end{equation*}

QML supplies functions to generate the most basic kernels (E.g. Gaussian, Laplacian). In the exercise below, we calculate a Gaussian kernel for the QM7 dataset.
In order to save time you can import the entire QM7 dataset as  <font color=red>Compound</font> objects from the file  <font color=red>tutorial_data.py</font> found in the tutorial GitHub repository.

In [0]:
# Import QM7, already parsed to QML
#from tutorial.tutorial_data import compounds

# For every compound generate a coulomb matrix or BoB
for mol in compounds:

    mol.generate_coulomb_matrix(size=50, sorting="row-norm")
    # mol.generate_bob(size=23, asize={"O":3, "C":7, "N":3, "H":16, "S":1})

# Make a big 2D array with all the representations
X = np.array([mol.representation for mol in compounds])

# Print all representations
print(X)
n=4000
n=4794
# Run on only a subset of the first 1000 (for speed)
X = X[:n]

# Define the kernel width
sigma = 1000.0

# K is also a Numpy array
K = gaussian_kernel(X, X, sigma)

# Print the kernel
print(K)

[[ 73.51669472  39.9972106   53.3587074  ...   0.           0.
    0.        ]
 [448.79438598  11.61622384  73.51669472 ...   0.           0.
    0.        ]
 [ 73.51669472  11.81274561  73.51669472 ...   0.           0.
    0.        ]
 ...
 [ 73.51669472  28.0779848   73.51669472 ...   0.           0.
    0.        ]
 [388.02344103  87.90710187  73.51669472 ...   0.           0.
    0.        ]
 [ 53.3587074   32.56476647  36.8581052  ...   0.           0.
    0.        ]]
[[1.         0.90604559 0.99403093 ... 0.94358486 0.89777439 0.89989695]
 [0.90604559 1.         0.90449117 ... 0.96497831 0.97654598 0.9622612 ]
 [0.99403093 0.90449117 1.         ... 0.94194341 0.89700917 0.89936837]
 ...
 [0.94358486 0.96497831 0.94194341 ... 1.         0.95671555 0.95203961]
 [0.89777439 0.97654598 0.89700917 ... 0.95671555 1.         0.95760516]
 [0.89989695 0.9622612  0.89936837 ... 0.95203961 0.95760516 1.        ]]


## Exercise 3: Regression

With the kernel matrix and representations sorted out in the previous two exercise, we can now solve the 

\begin{equation*}
\boldsymbol{\alpha}
\end{equation*} 

regression coefficients:

\begin{equation*}
\boldsymbol{\alpha} = (\mathbf{K} + \lambda \mathbf{I})^{-1} \mathbf{y}
\end{equation*}

One of the most efficient ways of solving this equation is using a Cholesky-decomposition.
QML includes a function named <font color=red>cho_solve()</font> to do this via the math module <font color=red>qml.math</font>.
In this step it is convenient to only use a subset of the full dataset as training data (see below).
The following builds on the code from the previous step.
To save time, you can import the PBE0/def2-TZVP atomization energies for the QM7 dataset from the file <font color=red>tutorial_data.py</font>.
This has been sorted to match the ordering of the representations generated in the previous exercise.
Extend your code from the previous step with the code below:


In [0]:
from qml_dr_obabel import energy_pbe0

# Assign 1000 first molecules to the training set

X_training = X[:n]
Y_training = energy_pbe0[:n]

sigma = 4000.0
K = gaussian_kernel(X_training, X_training, sigma)
print(K)

# Add a small lambda to the diagonal of the kernel matrix
K[np.diag_indices_from(K)] += 1e-8

print(len(K))
print(len(Y_training))


# Use the built-in Cholesky-decomposition to solve
alpha = cho_solve(K, Y_training)

print(alpha)

[[1.         0.99385237 0.99962589 ... 0.99637727 0.99328288 0.99342949]
 [0.99385237 1.         0.99374572 ... 0.99777438 0.99851776 0.99759855]
 [0.99962589 0.99374572 1.         ... 0.99626885 0.99322995 0.99339301]
 ...
 [0.99637727 0.99777438 0.99626885 ... 1.         0.99723825 0.99693292]
 [0.99328288 0.99851776 0.99322995 ... 0.99723825 1.         0.99729618]
 [0.99342949 0.99759855 0.99339301 ... 0.99693292 0.99729618 1.        ]]
4794
4794
[-10613872.26396403   2710438.7870555  -41759779.67568721 ...
 -87904899.25142123   -699698.0908894   -1824947.99455947]


## Exercise 4: Prediction

With the

\begin{equation*}
\boldsymbol{\alpha}
\end{equation*}

regression coefficients from the previous step, we have (successfully) trained the machine, and we are now ready to do predictions for other compounds.
This is done using the following equation:
    
\begin{equation*}
y\left(\widetilde{\mathbf{X}} \right) = \sum_i \alpha_i \  K\left( \widetilde{\mathbf{X}}, \mathbf{X}_i\right)
\end{equation*}

In this step we further divide the dataset into a training and a test set. Try using the last 1000 entries as test set.

In [0]:
# Assign 1000 last molecules to the test set
m=len(compounds)-n
X_test = X[-m:]
Y_test = energy_pbe0[-m:]

# calculate a kernel matrix between test and training data, using the same sigma
Ks = gaussian_kernel(X_test, X_training, sigma)

# Make the predictions
Y_predicted = np.dot(Ks, alpha)
print(Y_predicted.shape,)
# Calculate mean-absolute-error (MAE):
print(np.mean(np.abs(Y_predicted - Y_test)))

(1198,)
2.5392656968437994


## Exercise 5: Learning curves

Repeat the prediction from Exercise 2.4 with training set sizes of 1000, 2000, and 4000 molecules.

Note the MAE for every training size.

Plot a learning curve of the MAE versus the training set size.

Generate a learning curve for the Gaussian and Laplacian kernels, as well using the coulomb matrix and bag-of-bonds representations.

Which combination gives the best learning curve? Note you will have to adjust the kernel width (sigma) underway.

In [0]:
2.5392656968437994 pKa

## Exercise 6: Delta learning

A powerful technique in machine learning is the delta learning approach. Instead of predicting the PBE0/def2-TZVP atomization energies, we shall try to predict the difference between DFTB3 (a semi-empirical quantum method) and PBE0 atomization energies.
Instead of importing the <font color=red>energy_pbe0</font> data, you can import the <font color=red>energy_delta</font> and use this instead

In [0]:
from tutorial_data import energy_delta

Y_training = energy_delta[:1000]
Y_test = energy_delta[-1000:]

Finally re-draw one of the learning curves from the previous exercise, and note how the prediction improves.

In [0]:
# Import QM7, already parsed to QML
#from tutorial.tutorial_data import compounds

# For every compound generate a coulomb matrix or BoB
for mol in compounds:

    mol.generate_coulomb_matrix(size=50, sorting="row-norm")
    # mol.generate_bob(size=23, asize={"O":3, "C":7, "N":3, "H":16, "S":1})

# Make a big 2D array with all the representations
X = np.array([mol.representation for mol in compounds])

# Print all representations
print(X)
n=4000
n=4794
# Run on only a subset of the first 1000 (for speed)
X = X[:n]

# Define the kernel width
sigma = 1000.0

# K is also a Numpy array
K = laplacian_kernel(X, X, sigma)

# Print the kernel
print(K)

from qml_dr_obabel import energy_pbe0

# Assign 1000 first molecules to the training set

X_training = X[:n]
Y_training = energy_pbe0[:n]

sigma = 4000.0
K = laplacian_kernel(X_training, X_training, sigma)
print(K)

# Add a small lambda to the diagonal of the kernel matrix
K[np.diag_indices_from(K)] += 1e-8

print(len(K))
print(len(Y_training))


# Use the built-in Cholesky-decomposition to solve
alpha = cho_solve(K, Y_training)

print(alpha)

# Assign 1000 last molecules to the test set
m=len(compounds)-n
X_test = X[-m:]
Y_test = energy_pbe0[-m:]

# calculate a kernel matrix between test and training data, using the same sigma
Ks = laplacian_kernel(X_test, X_training, sigma)

# Make the predictions
Y_predicted = np.dot(Ks, alpha)
print(Y_predicted.shape,)
# Calculate mean-absolute-error (MAE):
print(np.mean(np.abs(Y_predicted - Y_test)))

[[ 73.51669472  39.9972106   53.3587074  ...   0.           0.
    0.        ]
 [448.79438598  11.61622384  73.51669472 ...   0.           0.
    0.        ]
 [ 73.51669472  11.81274561  73.51669472 ...   0.           0.
    0.        ]
 ...
 [ 73.51669472  28.0779848   73.51669472 ...   0.           0.
    0.        ]
 [388.02344103  87.90710187  73.51669472 ...   0.           0.
    0.        ]
 [ 53.3587074   32.56476647  36.8581052  ...   0.           0.
    0.        ]]
[[1.         0.02466138 0.44175017 ... 0.33631362 0.00949206 0.00701722]
 [0.02466138 1.         0.02130759 ... 0.02861298 0.0447762  0.03589143]
 [0.44175017 0.02130759 1.         ... 0.27990503 0.00887682 0.00644736]
 ...
 [0.33631362 0.02861298 0.27990503 ... 1.         0.012205   0.0118404 ]
 [0.00949206 0.0447762  0.00887682 ... 0.012205   1.         0.0183275 ]
 [0.00701722 0.03589143 0.00644736 ... 0.0118404  0.0183275  1.        ]]
[[1.         0.396282   0.81525633 ... 0.76152842 0.31213331 0.28942845]
 [0